In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Varshil\Downloads\comments_data (1).csv")

In [3]:
df.head()

,CommentID,Timestamp,Username,Comment,Date,Likes,reply_comment_id,replies
0,Ugz9iIF4cwCpLJd-Jl54AaABAg,2024-07-15T16:13:38Z,@adityasaha4704,I thank you for my very existence,2024-07-15T16:13:38Z,0,NaN,NaN
1,UgxnDVp1OB0v5Wf8iD54AaABAg,2023-09-18T19:14:31Z,@tippydippy6529,every couple of weeks I find myself watching t...,2023-09-18T19:14:30Z,2,NaN,NaN
2,Ugyw_I56xAhWu9LctcV4AaABAg,2023-07-17T16:54:37Z,@RamonSBK,Vagabond Reminds me everyday that anything is ...,2023-09-07T12:22:47Z,0,NaN,NaN
3,UgxWtjYys9HNS7ZxRap4AaABAg,2023-07-31T16:22:14Z,@wubbalubbadubdub777,ayahviv ! oued amizour days a rayan !,2023-07-31T16:22:14Z,0,NaN,NaN
4,Ugwu8Jy2t39SzRnUjoV4AaABAg,2023-07-19T23:03:47Z,@angelagrullon2189,La vibes que transmite esta cancion mierda.......,2023-07-19T23:03:47Z,0,NaN,NaN


In [13]:
import regex # re2 for faster matching and replacing or removing
import nltk
import emoji

def sent_process(sent):
    words = nltk.word_tokenize(sent)
    for i, word in enumerate(words):
        # Remove various types of URLs
        word = word.lower()
        word = regex.sub(r'\p{Emoji}', lambda m: emoji.demojize(m.group()), word)
        word = regex.sub(r'@[A-Za-z0-9]+\b', '', word)  # rplace usernames from the comments
        word = regex.sub(r'https?://\S+', '', word)  # Remove HTTP/HTTPS links
        word = regex.sub(r'[^a-zA-Z0-9\s]', ' ', word)
        words[i] = word.strip()
        
    return " ".join(words)
    
def text_process(df):
    df['cleaned_parent_comm'] = [sent_process(x) for x in df.Comment]
    #df['cleaned_replies'] = [" || ".join([sent_process(y) for y in x.split(" || ") if x else np.nan]) for x in df.replies]

    df['cleaned_replies'] = df['replies'].apply(
        lambda x: " || ".join([sent_process(y) for y in x.split(" || ")]) if pd.notna(x) else np.nan
    )

    return df

df = text_process(df)

In [14]:
df.head()

,CommentID,Timestamp,Username,Comment,Date,Likes,reply_comment_id,replies,cleaned_parent_comm,cleaned_replies
0,Ugz9iIF4cwCpLJd-Jl54AaABAg,2024-07-15T16:13:38Z,@adityasaha4704,I thank you for my very existence,2024-07-15T16:13:38Z,0,NaN,NaN,i thank you for my very existence,NaN
1,UgxnDVp1OB0v5Wf8iD54AaABAg,2023-09-18T19:14:31Z,@tippydippy6529,every couple of weeks I find myself watching t...,2023-09-18T19:14:30Z,2,NaN,NaN,every couple of weeks i find myself watching t...,NaN
2,Ugyw_I56xAhWu9LctcV4AaABAg,2023-07-17T16:54:37Z,@RamonSBK,Vagabond Reminds me everyday that anything is ...,2023-09-07T12:22:47Z,0,NaN,NaN,vagabond reminds me everyday that anything is ...,NaN
3,UgxWtjYys9HNS7ZxRap4AaABAg,2023-07-31T16:22:14Z,@wubbalubbadubdub777,ayahviv ! oued amizour days a rayan !,2023-07-31T16:22:14Z,0,NaN,NaN,ayahviv oued amizour days a rayan,NaN
4,Ugwu8Jy2t39SzRnUjoV4AaABAg,2023-07-19T23:03:47Z,@angelagrullon2189,La vibes que transmite esta cancion mierda.......,2023-07-19T23:03:47Z,0,NaN,NaN,la vibes que transmite esta cancion mierda y ...,NaN


### 3 types of data we need : 
#### 1. all comments ==> topic modelling for top discussed topics
#### 2. all comments ==> sentiment analysis
#### 3. top 5 comments and replies ==> summarization

In [15]:
all_text = list(df.cleaned_parent_comm) +  list(df.cleaned_replies)

In [16]:
top_comments_df = df.sort_values(by = 'Likes', ascending = False).reset_index()
top_comments_df.head()

,index,CommentID,Timestamp,Username,Comment,Date,Likes,reply_comment_id,replies,cleaned_parent_comm,cleaned_replies
0,190,Ugzac0O_LaZWK1ZtsCd4AaABAg,2021-07-10T11:23:55Z,@MR.X-l2u,Vegabond and Berserk are the two best manga ev...,2021-07-10T11:23:55Z,2892,|| UgxnDVp1OB0v5Wf8iD54AaABAg || UgxWtjYys9HN...,|| every couple of weeks I find myself watchi...,vegabond and berserk are the two best manga ev...,|| every couple of weeks i find myself watchi...
1,180,UgzyQhJPJ-zHhKkDN0B4AaABAg,2021-08-30T17:34:52Z,@cheeseman2219,Musashi vs 70 Yoshioka men was the most memora...,2021-08-30T17:34:52Z,1176,|| UgztSsIAWzxaZHhlgYx4AaABAg || UgwudFWY-fxY...,|| You have to walk through the depths of Hel...,musashi vs 70 yoshioka men was the most memora...,|| you have to walk through the depths of hel...
2,192,Ugx_LdgI0vWOumWYS_d4AaABAg,2021-07-10T05:49:27Z,@abandonaccount8582,Vegabond is underrated more people should read...,2021-07-10T05:49:27Z,1158,|| UgwbxjfdeS8E9KEuCsl4AaABAg || Ugwl4x2FegQ3...,|| This man is my inspiration 💯🔥 || what a gr...,vegabond is underrated more people should read...,|| this man is my inspiration hundred points ...
3,161,UgypwPJSMIyeogONpxl4AaABAg,2021-10-27T10:47:26Z,@DavionX13,"Preoccupied with a single leaf, you won't see ...",2021-10-27T10:47:26Z,928,|| Ugz9iIF4cwCpLJd-Jl54AaABAg || Ugx4ws8nBg-k...,|| I thank you for my very existence || he is...,preoccupied with a single leaf you wo n t see...,|| i thank you for my very existence || he is...
4,186,UgxMNe48oT4QElg2klN4AaABAg,2021-07-21T01:06:14Z,@sovereignrepublicofcopticx6397,The best part is that he was real.,2021-07-21T01:06:14Z,913,|| Ugxm-p8LiLGYRbc0wFd4AaABAg || UgxYCUvdzw2a...,|| The fact that beardless/young Musashi's fa...,the best part is that he was real,|| the fact that beardless young musashi s fa...


In [17]:
top_comments = []
for i in range(5):
    dic = {}
    dic['comment'] = top_comments_df.loc[i, "cleaned_parent_comm"]
    dic['replies'] = top_comments_df.loc[i, "cleaned_replies"]
    dic['likes'] = top_comments_df.loc[i, "Likes"]
    top_comments.append(dic)
top_comments[0]

{'comment': 'vegabond and berserk are the two best manga ever made ',
 'replies': ' || every couple of weeks i find myself watching this on repeat  have for a couple of years now    great job || ayahviv  oued amizour days a rayan  || la vibes que transmite esta cancion mierda  y mas con este manga que es una verdadera grasa || vagabond reminds me everyday that anything is possible and when people say it is n t it means it s not possible for them because they feel like it isnt || when musashi said  i am vagabond   i felt that  || big sad || how to download english version pdg for free || thankyou i will use these clips for my edit and i will give credit on tiktok || les meilleurs planches se berserk et vagabond || a good read smiling face with smiling eyes || just waiting to watch the insanely animated version of  vagabond  || eu tbm  depois que eu conhe i esse manga minha vida mudo por completo || witch remix is this  ||   uzochiokeke4328 lmaoooo ||   josephisrael507 i was just trying 

In [18]:
len(top_comments)

5

### WE HAVE DATA NOW

### TOPIC MODELLING DOESNT CARE ABOUT THE START AND END OF SENTENCE SINCE IT USES BAG OF WORDS FOR MODELLING T`OPICS

In [19]:
if np.nan: print(True)

True


In [30]:
all_text[0]

'i thank you for my very existence'

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = []
for sent in all_text:
    if type(sent) == float: continue
    
    # Tokenize the text
    words = word_tokenize(sent)  # Convert to lowercase for consistency
    
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]
    
    # Join words back into a sentence
    text.append(" ".join(filtered_words))

In [34]:
text[0]

'thank existence'

In [15]:
!pip install top2vec

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified




  Using cached top2vec-1.0.36-py3-none-any.whl.metadata (22 kB)
  Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached umap_learn-0.5.7-py3-none-any.whl.metadata (21 kB)
  Using cached hdbscan-0.8.40-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached wordcloud-1.9.4-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached numba-0.61.0-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached llvmlite-0.44.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached top2vec-1.0.36-py3-none-any.whl (33 kB)
Using cached gensim-4.3.3-cp312-cp312-win_amd64.whl (24.0 MB)
Using cached hdbscan-0.8.40-cp312-cp312-win_amd64.whl (726 kB)
Using cached umap_learn-0.5.7-py3-none-any.whl (88 kB)
Using cached wordcloud-1.9.4-cp312-cp312-win_amd64.whl (301 kB)
Using cached numba-0.61.0-cp312-cp312-win_amd64.whl (2.8 MB)
Usin

In [ ]:
"""from bertopic import BERTopic

# Example: Unstructured text (Reddit comments, YouTube comments, news, etc.)
docs = [
    "Bitcoin is going to the moon! Crypto is booming.",
    "Ethereum is the future of blockchain technology.",
    "I love playing football on weekends.",
    "Messi and Ronaldo are the greatest footballers ever.",
    "Stock market is very volatile these days.",
    "Investing in cryptocurrency can be risky but rewarding."
]

# Initialize and train BERTopic
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

# Show most discussed topics
print(topic_model.get_topic_info())

==> OSError: [WinError 182] The operating system cannot run %1. Error loading "C:\Users\Varshil\miniconda3\envs\NLP-Tx\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

"""

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [23]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.decomposition import PCA
import hdbscan
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

# Load Hugging Face Transformer model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Varshil\miniconda3\envs\GANLAB\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Varshil\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

ValueError: n_components=50 must be between 0 and min(n_samples, n_features)=10 with svd_solver='full'

In [36]:
len(text)

244

In [37]:

# 2. Generate Embeddings with Hugging Face Transformers
def get_transformer_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Use [CLS] token embedding
    return embeddings

embeddings = get_transformer_embeddings(text)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3070230528 bytes.

In [25]:
embeddings.shape

(10, 768)

In [ ]:
# 3. Reduce Dimensionality
pca = PCA(n_components=50)  # Reduce from 768D to 50D for better clustering
reduced_embeddings = pca.fit_transform(embeddings)

# 4. Apply Clustering (HDBSCAN for adaptive topic discovery)
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean')
labels = clusterer.fit_predict(reduced_embeddings)

# 5. Assign Topics to Clusters (Nearest Neighbors for Top Words)
def extract_top_keywords(embeddings, docs, labels, n_neighbors=3):
    unique_labels = set(labels)
    topic_keywords = {}
    
    for label in unique_labels:
        if label == -1:  # Ignore noise points
            continue
        
        cluster_docs = [docs[i] for i in range(len(docs)) if labels[i] == label]
        cluster_embeddings = [embeddings[i] for i in range(len(docs)) if labels[i] == label]
        
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        knn.fit(cluster_embeddings)
        distances, indices = knn.kneighbors(cluster_embeddings)
        
        topic_keywords[label] = [cluster_docs[i] for i in indices[0]]
    
    return topic_keywords

# Extract keywords
topics = extract_top_keywords(reduced_embeddings, documents, labels)

# 6. Visualize Clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=reduced_embeddings[:, 0], y=reduced_embeddings[:, 1], hue=labels, palette='tab10')
plt.title("Topic Clusters")
plt.show()

# Print Topics
for topic, keywords in topics.items():
    print(f"Topic {topic}: {keywords}")